In [1]:
import time
import pyspark
import numpy as np
import pandas as pd
import itertools as it
from scipy import stats
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
try:
    sc.stop()
except Exception as e:
    print(e)
try:
    sessn.stop()
except Exception as e:
    print(e)
try:
    sc.SparkContext.stop()
except Exception as e:
    print(e)

name 'sessn' is not defined
'SparkContext' object has no attribute 'SparkContext'


In [3]:
def get_md(u, v, IV):
    """
    Compute the Mahalanobis distance (variance).
    The Mahalanobis distance between  `u` and `v`, is defined as
    .. math::{ (u-v) VI (u-v)^T }
    
    Parameters
    ----------
    u : (N,) array_like
        Input array.
    v : (N,) array_like
        Input array.
    VI : ndarray
        The inverse of the covariance matrix.

    Returns
    -------
    mahalanobis : double
        The Mahalanobis distance between vectors `u` and `v`.
        
    """
    u_v = u - v 
    lpart = np.dot(u_v, IV)
    fpart = np.dot(lpart, u_v.T)
    return fpart

In [4]:
def md_spark_df(scoring_df, invcovar_df, mean_df, col_list):
    try:
        scoring_df = scoring_df.select(["servicing_npi", "srk_clm_id_rj"] + col_list)
        ms_sprk = scoring_df.join(mean_df, how ="inner", on ="servicing_npi")
        ms_sprk = ms_sprk.cache()
        ###### u - v ######
        for s in col_list:
            s_m = s + "_m"
            s_avg = "avg_{}".format(s)
            ms_sprk = ms_sprk.withColumn(s_m, F.col(s) - F.col(s_avg))
            ms_sprk = ms_sprk.drop(s)
            ms_sprk = ms_sprk.drop(s_avg)
        ###### {u-v} * INV(covar) : Left Part ######
        ms_sprk = ms_sprk.join(invcovar_df, how ="inner", on ="servicing_npi")
        for s in col_list:
            mul_list_d = []
            s_lp = s + "_lp"
            for s2 in col_list:
                s_m = s2 + "_m"
                s_covar = "covar_samp_{}_{}".format(s, s2)
                s_m_covar_mul = s2 + "_m_covar_mul"
                ms_sprk = ms_sprk.withColumn(s_m_covar_mul, F.col(s_m)*F.col(s_covar))
                mul_list_d.append(s_m_covar_mul)
                ms_sprk = ms_sprk.drop(s_covar)
            mul_list = [F.col(x) for x in mul_list_d]
            # For All Signals - Claim Score
            if len(mul_list) == 11:
                ms_sprk = ms_sprk.withColumn(s_lp, mul_list[0] + mul_list[1] + mul_list[2] + mul_list[3] + mul_list[4] 
                                             + mul_list[5] + mul_list[6] + mul_list[7] + mul_list[8] + mul_list[9] + 
                                             mul_list[10])
            # For Reason Code
            if len(mul_list) == 10:
                ms_sprk = ms_sprk.withColumn(s_lp, mul_list[0] + mul_list[1] + mul_list[2] + mul_list[3] + mul_list[4] 
                                             + mul_list[5] + mul_list[6] + mul_list[7] + mul_list[8] + mul_list[9])
            for col_ in mul_list_d:
                ms_sprk = ms_sprk.drop(col_)
        ###### {{u-v} * INV(covar)} * {u-v}.T ###### 
        mul_list_d = []
        for s in col_list:
            s_lp = s + "_lp"
            s_m = s + "_m"
            s_lp_m_mul = s + "_lp_m_mul"
            ms_sprk = ms_sprk.withColumn(s_lp_m_mul, F.col(s_lp)*F.col(s_m))
            ms_sprk = ms_sprk.drop(s_lp)
            ms_sprk = ms_sprk.drop(s_m)
            mul_list_d.append(s_lp_m_mul)
        mul_list = [F.col(x) for x in mul_list_d]
        # For All Signals - Claim Score
        if len(mul_list) == 11:
            ms_sprk = ms_sprk.withColumn("md", mul_list[0] + mul_list[1] + mul_list[2] + mul_list[3] + mul_list[4] 
                                             + mul_list[5] + mul_list[6] + mul_list[7] + mul_list[8] + mul_list[9] + 
                                             mul_list[10])
            # Taking the absolute 
            ms_sprk = ms_sprk.withColumn("md", F.abs(F.col("md")))
            ms_sprk = ms_sprk.withColumn("md", F.round(F.sqrt(F.col("md")), 3))
        # For Reason Code
        if len(mul_list) == 10:
            ms_sprk = ms_sprk.withColumn("md_s", mul_list[0] + mul_list[1] + mul_list[2] + mul_list[3] + mul_list[4] 
                                             + mul_list[5] + mul_list[6] + mul_list[7] + mul_list[8] + mul_list[9])
            # Taking the absolute 
            ms_sprk = ms_sprk.withColumn("md_s", F.abs(F.col("md_s")))
            ms_sprk = ms_sprk.withColumn("md_s", F.round(F.sqrt(F.col("md_s")), 3))
        for col_ in mul_list_d:
            ms_sprk = ms_sprk.drop(col_)
        
        return ms_sprk
    except Exception as e:
        print(e) 

In [5]:
def get_reason_code(claim_):
    # Generating Signals for a Claim
    snpi = claim_[0]
    # Extracting signal effect from claim
    claim_se = claim_[3:].astype(float)
    ## Figuring out Reason for high MD score
    # Sorting the effect
    claim_sort  = np.flip(np.argsort(claim_se))
    claim_se_thi = np.argwhere(claim_se > th).flatten()
    claim_se_thi = claim_se_thi[0:4]
    reason_c_list = []
    for rnk in claim_sort:
        if rnk in claim_se_thi:
            reason_c_list.append(mapper[rnk])
    reasn_desc_list = []
    for rc in reason_c_list:
        rc_avg = "avg_" + rc
        try:
            m_value = str(np.round(mean_data_rdesc_dict[snpi][rc_avg], 2))
            reasn_desc_list.append(reason_desc[rc].replace("(&avg&)", m_value))
        except:
            reasn_desc_list.append(reason_desc[rc])
    return [snpi, claim_[1], claim_[2]] + reasn_desc_list

In [6]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '4g'), 
                                   ('spark.executor.instances', '8'),
                                   ('spark.executor.cores', '4'), 
                                   ('spark.driver.cores','4'), 
                                   ('spark.driver.memory','4g'),
                                   ('spark.yarn.queue', 'araadh_q1.arapi_sq1'),
                                   ("spark.serializer", "org.apache.spark.serializer.KryoSerializer"),
                                  ('spark.yarn.executor.memoryOverhead', '1g')])
sessn = SparkSession.builder.appName("Scoring").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
sessn

#### Reading the Data

In [7]:
# Input Tables
s_table = "pi_usrs.hpp_ps_signal_data_scoring"
mean_table = "pi_usrs.hpp_ps_mean_data"
icov_table = "pi_usrs.hpp_ps_icov_data"
# Output Tables
flagged_table = "pi_usrs.hpp_ps_flagged_clm_scoring"
reason_code_table = "pi_usrs.hpp_ps_flagged_clm_scoring_effect_"

In [8]:
signal_list =  ['billed_amt_sum',
                 'clm_ln_cnt',
                 'clm_unit_billed',
                 'proc_code_unusual',
                 'pos_unsual',
                 'em_proc_ind',
                 'mod91_ind',
                 'modTC_ind',
                 'mod59_ind',
                 'high_cst_proc_ind',
                 'emgncy_ind']

In [9]:
scoring_data_temp = sessn.table(s_table)
mean_data = sessn.table(mean_table)
icovariance_data = sessn.table(icov_table)

In [10]:
scoring_data_temp_pd = scoring_data_temp.toPandas()
mean_data_pd = mean_data.toPandas()
icov_op_pd = icovariance_data.toPandas()

### Claim Scoring

#### Imputation

In [11]:
scoring_imputed_data = scoring_data_temp.join(mean_data, on = "servicing_npi", how = "inner")

In [12]:
# Imputing the values lesser or equal to mean with mean
for s in ['billed_amt_sum', 'clm_ln_cnt', 'clm_unit_billed']:
    s_avg = "avg_" + s
    s_i = "imp_" + s
    scoring_imputed_data = scoring_imputed_data.withColumn(s_i, F.when((F.col(s) <= F.col(s_avg)), F.col(s_avg)).otherwise(F.col(s)))
    scoring_imputed_data = scoring_imputed_data.drop(s)
    
# Imputing the values greater or equal to mean with mean
for s in ['proc_code_unusual', 'pos_unsual']:
    s_avg = "avg_" + s
    s_i = "imp_" + s
    scoring_imputed_data = scoring_imputed_data.withColumn(s_i, F.when((F.col(s) >= F.col(s_avg)), F.col(s_avg)).otherwise(F.col(s)))
    scoring_imputed_data = scoring_imputed_data.drop(s)
    
imp_col = [c for c in scoring_imputed_data.columns if "imp_" in c]
avg_col = [c for c in scoring_imputed_data.columns if "avg_" in c]

for c in avg_col:
    scoring_imputed_data = scoring_imputed_data.drop(c)
    
for c_i in imp_col:
    c = c_i.replace("imp_", "")
    scoring_imputed_data = scoring_imputed_data.withColumnRenamed(c_i, c)
    
scoring_imputed_data = scoring_imputed_data.select(['servicing_npi', 'srk_clm_id_rj'] + signal_list)

In [13]:
scoring_imputed_data = scoring_imputed_data.cache()

#### Claim Score ( Mahalanobis Distance )

In [49]:
scoring_clm_md = md_spark_df(scoring_imputed_data, icovariance_data, mean_data, signal_list)

In [50]:
scoring_clm_md_pd = scoring_clm_md.toPandas()
scoring_clm_md_pd.head(10)

servicing_npi               srk_clm_id_rj     md
0    1003127457  202002070390001900000MJ11X  1.638
1    1003127457  202002240850000500000MIDKE  1.173
2    1003865213  202002040430000900000MIBKA  0.316
3    1003865213  202002260060002600000MJONP  0.316
4    1003866393  202002270920001000000MIRU8  2.799
5    1003882523  202003021060000700000MIQV9  0.872
6    1003892779  202003300290000300000MIC65  1.369
7    1013065580  202002270400000600000MKJHZ  1.743
8    1013065580  202003090280001300000MIH2I  3.502
9    1013065580  202003100310000600000MIF39  1.822

#### Anomalous Claim Flagging

##### Sensitivity Analysis to determine the critical value

In [16]:
md_des = scoring_clm_md_pd['md'].describe().reset_index()
md_des

index             md
0  count  168948.000000
1   mean      80.785939
2    std    1150.001310
3    min       0.000000
4    25%       1.024000
5    50%       1.631000
6    75%       2.766000
7    max  103007.792000

##### 3 Sigma rule -  Empirical Rule

In [17]:
def get_zscore(x, m, std):
    z = (x-m)/std
    return z

In [18]:
# Transform the data distribution to a normal distribution
md_np = scoring_clm_md_pd['md'].values + 0.0001
md_np_bxcx, lmbda = stats.boxcox(md_np)
scoring_clm_md_pd['md_boxcox'] = md_np_bxcx
m = scoring_clm_md_pd['md_boxcox'].mean()
std = scoring_clm_md_pd['md_boxcox'].std()
scoring_clm_md_pd['zscore'] = scoring_clm_md_pd['md_boxcox'].apply(lambda x : get_zscore(x, m, std))
ucl = 3
flagged_data_empirical = scoring_clm_md_pd[scoring_clm_md_pd['zscore'] >= ucl]
flagged_data_empirical.head(10)

servicing_npi               srk_clm_id_rj        md  md_boxcox    zscore
12     1013934983  202002130520000900000MIECP  6783.832   9.567884  4.604562
16     1023005204  202002240090002600000MIKXB  6326.256   9.485975  4.562008
33     1033536362  202003190020002500000MJBH7  2809.271   8.541403  4.071269
76     1083790729  202002220020000400000MJ9ZE  8012.127   9.763492  4.706188
229    1265415939  202003020710000100000MIBQ5  6654.698   9.545331  4.592846
237    1265479315  202003310270000400000MJKD4   572.590   6.730587  3.130485
274    1295768844  202002240060002100000MIIAS  3752.066   8.876526  4.245377
306    1326072448  202002031350000400000MIJAA   879.228   7.213730  3.381495
309    1326072448  202003061080001700000MIZOD   878.483   7.212771  3.380997
310    1326072448  202003240030001300000MJJCM   878.395   7.212658  3.380938

In [19]:
flg_clm_cnt = flagged_data_empirical.shape[0]
tlt_clm = scoring_clm_md_pd.shape[0]
print("{} claim flagged out of {}, which is {}%".format(flg_clm_cnt, tlt_clm,  np.round((flg_clm_cnt*100)/tlt_clm), 3))

3055 claim flagged out of 168948, which is 2.0%


In [20]:
flagged_data_empirical['md'].describe()

count      3055.000000
mean       4034.190079
std        7562.034707
min         459.496000
25%         878.340000
50%        1542.835000
75%        4438.916000
max      103007.792000
Name: md, dtype: float64

##### Tukey's Fence

In [21]:
Q1 = scoring_clm_md_pd['md'].quantile(0.25)
Q3 = scoring_clm_md_pd['md'].quantile(0.75)
IQR = Q3 - Q1
k = 3
ucl = Q3 + (k*IQR)
flagged_data_tukey = scoring_clm_md_pd[scoring_clm_md_pd['md'] >= ucl]
flagged_data_tukey.head(10)

servicing_npi               srk_clm_id_rj        md  md_boxcox    zscore
10    1013065580  202003160560000700000MIN6G   110.594   4.912678  2.186017
12    1013934983  202002130520000900000MIECP  6783.832   9.567884  4.604562
16    1023005204  202002240090002600000MIKXB  6326.256   9.485975  4.562008
24    1033119540  202003250530001500000MIKQQ     8.154   2.138997  0.744991
28    1033213236  202002270130002300000MJS7F    12.752   2.605423  0.987315
32    1033536362  202003190020002100000MJ72Q    43.500   3.904933  1.662457
33    1033536362  202003190020002500000MJBH7  2809.271   8.541403  4.071269
56    1063632149  202003031400000700000MIX1S   200.547   5.564444  2.524632
57    1063632149  202003261480002200000MIN06    16.944   2.903856  1.142362
60    1063912814  202003020940000400000MIK6B   413.170   6.365496  2.940808

In [22]:
flg_clm_cnt = flagged_data_tukey.shape[0]
tlt_clm = scoring_clm_md_pd.shape[0]
print("{} claim flagged out of {}, which is {}%".format(flg_clm_cnt, tlt_clm,  np.round((flg_clm_cnt*100)/tlt_clm), 3))

19418 claim flagged out of 168948, which is 11.0%


In [23]:
flagged_data_tukey['md'].describe()

count     19418.000000
mean        689.088751
std        3330.008610
min           7.993000
25%          14.222000
50%          37.567500
75%         156.702750
max      103007.792000
Name: md, dtype: float64

##### By an arbitrary number 

In [24]:
ucl = 200
flagged_data_cutoff = scoring_clm_md_pd[scoring_clm_md_pd['md'] >= ucl]
flagged_data_cutoff.head(10)

servicing_npi               srk_clm_id_rj        md  md_boxcox    zscore
12     1013934983  202002130520000900000MIECP  6783.832   9.567884  4.604562
16     1023005204  202002240090002600000MIKXB  6326.256   9.485975  4.562008
33     1033536362  202003190020002500000MJBH7  2809.271   8.541403  4.071269
56     1063632149  202003031400000700000MIX1S   200.547   5.564444  2.524632
60     1063912814  202003020940000400000MIK6B   413.170   6.365496  2.940808
76     1083790729  202002220020000400000MJ9ZE  8012.127   9.763492  4.706188
85     1104860105  202002031620000400000MIE07   314.213   6.060834  2.782525
229    1265415939  202003020710000100000MIBQ5  6654.698   9.545331  4.592846
237    1265479315  202003310270000400000MJKD4   572.590   6.730587  3.130485
274    1295768844  202002240060002100000MIIAS  3752.066   8.876526  4.245377

In [25]:
flg_clm_cnt = flagged_data_cutoff.shape[0]
tlt_clm = scoring_clm_md_pd.shape[0]
print("{} claim flagged out of {}, which is {}%".format(flg_clm_cnt, tlt_clm,  np.round((flg_clm_cnt*100)/tlt_clm), 3))

4374 claim flagged out of 168948, which is 3.0%


In [26]:
flagged_data_cutoff['md'].describe()

count      4374.000000
mean       2902.536833
std        6550.157530
min         200.464000
25%         378.463000
50%         879.228000
75%        2934.666750
max      103007.792000
Name: md, dtype: float64

##### Chi Square Test

In [60]:
# Degree of Freedom = Number of Feature - 1
df_ = len(signal_list) - 1
sg_level = 0.01
scoring_clm_md_pd['mdD2'] = scoring_clm_md_pd['md'] * scoring_clm_md_pd['md']

In [61]:
# Critical values 
cv = stats.chi2.ppf((1-sg_level), df = df_)
flagged_data_chicv = scoring_clm_md_pd[scoring_clm_md_pd['mdD2']>cv]
print("Critical Value {}".format(cv))
flg_clm_cnt = flagged_data_chicv.shape[0]
tlt_clm = scoring_clm_md_pd.shape[0]
print("{} claim flagged out of {}, which is {}%".format(flg_clm_cnt, tlt_clm,  np.round((flg_clm_cnt*100)/tlt_clm), 3))
flagged_data_chicv['md'].describe()

Critical Value 23.209251158954356
25972 claim flagged out of 168948, which is 15.0%


count     25972.000000
mean        516.737997
std        2894.577321
min           4.819000
25%           7.915000
50%          18.257000
75%         103.608000
max      103007.792000
Name: md, dtype: float64

In [62]:
# Compute the P-Values
scoring_clm_md_pd['p_value'] = 1 - stats.chi2.cdf(scoring_clm_md_pd['mdD2'], df = df_)

# Extreme values with a significance level of 0.01
flagged_data_chipv = scoring_clm_md_pd[scoring_clm_md_pd['p_value'] < sg_level]

flg_clm_cnt = flagged_data_chipv.shape[0]
tlt_clm = scoring_clm_md_pd.shape[0]
print("{} claim flagged out of {}, which is {}%".format(flg_clm_cnt, tlt_clm,  np.round((flg_clm_cnt*100)/tlt_clm), 3))
flagged_data_chipv['md'].describe()

25972 claim flagged out of 168948, which is 15.0%


count     25972.000000
mean        516.737997
std        2894.577321
min           4.819000
25%           7.915000
50%          18.257000
75%         103.608000
max      103007.792000
Name: md, dtype: float64

##### Claim Flagging

In [71]:
scoring_clm_md = scoring_clm_md.withColumn("md2", F.col("md") * F.col("md"))
# Computing the critical Value
df_ = len(signal_list) - 1
sg_level = 0.01
cv = stats.chi2.ppf((1-sg_level), df = df_)
print("Critical Value {}".format(cv))
flagged_data = scoring_clm_md.filter(F.col("md2") > cv)

Critical Value 23.209251158954356


#### Reason Code Generation

In [72]:
group_level = ["servicing_npi"]

In [73]:
flagged_data_scoring = scoring_imputed_data.join(flagged_data, on = ["servicing_npi", "srk_clm_id_rj"], how = "inner")
flagged_data_scoring.write.mode("overwrite").format("orc").saveAsTable(flagged_table)

In [74]:
flagged_data_scoring = sessn.table(flagged_table)

In [75]:
for s in signal_list:
    try:
        print("Calculating {} effect".format(s))
        signal_effect = s + "_effect"
        signal_list_r = signal_list.copy()
        signal_list_r.remove(s)
        mean_list_r = ["avg_{}".format(r) for r in signal_list_r]
        mean_data_r = mean_data.select(group_level + mean_list_r)
        signal_list_product_r = list(it.product(signal_list_r, repeat =2))
        icov_list_r = ["covar_samp_{}_{}".format(r[0], r[1]) for r in signal_list_product_r]
        icovariance_data_r = icovariance_data.select(group_level + icov_list_r)
        md_r_op = md_spark_df(flagged_data_scoring, icovariance_data_r, mean_data_r, signal_list_r)
        md_r_op = md_r_op.withColumnRenamed("md_s", signal_effect)
        md_r_op = md_r_op.drop("md")
        r_op_table = reason_code_table + s
        md_r_op.write.mode("overwrite").format("orc").saveAsTable(r_op_table)
    except:
        print("Error occur at {}".format(s))

Calculating billed_amt_sum effect
Calculating clm_ln_cnt effect
Calculating clm_unit_billed effect
Calculating proc_code_unusual effect
Calculating pos_unsual effect
Calculating em_proc_ind effect
Calculating mod91_ind effect
Calculating modTC_ind effect
Calculating mod59_ind effect
Calculating high_cst_proc_ind effect
Calculating emgncy_ind effect


In [76]:
for s in signal_list:
    r_op_table = reason_code_table + s
    md_r_effect = sessn.table(r_op_table)
    flagged_data_scoring = flagged_data_scoring.join(md_r_effect, on = ["servicing_npi", "srk_clm_id_rj"], how = "inner")

In [81]:
flagged_data_scoring_pd = flagged_data_scoring.toPandas()

##### Top 4 Reason Code Calculation

In [82]:
mapper = {}
for v in range(len(signal_list)):
    mapper[v] = signal_list[v]

In [83]:
mean_data_rdesc = mean_data_pd[['servicing_npi', 'avg_billed_amt_sum', 'avg_clm_ln_cnt','avg_clm_unit_billed', 'avg_proc_code_unusual', 'avg_pos_unsual']]
mean_data_rdesc_dict = mean_data_rdesc.set_index("servicing_npi").T.to_dict()

In [91]:
reason_desc = {'billed_amt_sum': 'Billed Amount of claim is higher than expected. Average Value = (&avg&)',
               'clm_ln_cnt': 'No. of claim lines are higher than expected. Average Value = (&avg&)',
               'clm_unit_billed': 'No. of units billed is higher than expected. Average Value = (&avg&)',
               'em_proc_ind': 'E&M code is used in claim.',
               'emgncy_ind': "Claim without an Emergency Indicator was performed at 'Emergency' place of service",
               'high_cst_proc_ind': 'High cost J procedure is used in claim.',
               'mod59_ind': 'Modifier 59 is used in claim',
               'mod91_ind': 'Modifier 91 is used in claim',
               'modTC_ind': 'Modifier TC is used in claim',
               'pos_unsual': 'An unusual place of service is observed. Used (&avg&) only in past 3 months',
               'proc_code_unusual': 'An unusual procedure code is observed. Used (&avg&) only in past 3 months'}

In [92]:
col_effect = [c for c in flagged_data_scoring_pd.columns if "_effect" in c]
# Percentage deviation of different signal w.r.t Md
diff_pd = pd.DataFrame()
diff_pd['servicing_npi'] = flagged_data_scoring_pd['servicing_npi']
diff_pd['srk_clm_id_rj'] = flagged_data_scoring_pd['srk_clm_id_rj']
diff_pd['md'] = flagged_data_scoring_pd['md']
for c in col_effect:
    diff_pd[c] = (flagged_data_scoring_pd[c] - flagged_data_scoring_pd["md"])
    diff_pd[c] = diff_pd[c].abs() 
    diff_pd[c] = (diff_pd[c]*100)/flagged_data_scoring_pd['md']

In [93]:
diff_pd.head(10).T

0  \
servicing_npi                             1023005204   
srk_clm_id_rj             202002070390000200000MJ34B   
md                                           6326.25   
billed_amt_sum_effect                        99.9624   
clm_ln_cnt_effect                                  0   
clm_unit_billed_effect                             0   
proc_code_unusual_effect                   0.0145426   
pos_unsual_effect                                  0   
em_proc_ind_effect                        0.00273464   
mod91_ind_effect                                   0   
modTC_ind_effect                                   0   
mod59_ind_effect                                   0   
high_cst_proc_ind_effect                           0   
emgncy_ind_effect                                  0   

                                                   1  \
servicing_npi                             1023476819   
srk_clm_id_rj             202002110470000200000MID1H   
md                                              8.14   
billed_amt_sum_effect                              0   
clm_ln_cnt_effect                                  0   
clm_unit_billed_effect                             0   
proc_code_unusual_effect                     15.6265   
pos_unsual_effect                                  0   
em_proc_ind_effect                            82.113   
mod91_ind_effect                                   0   
modTC_ind_effect                                   0   
mod59_ind_effect                                   0   
high_cst_proc_ind_effect                           0   
emgncy_ind_effect                                  0   

                                                   2  \
servicing_npi                             1033186200   
srk_clm_id_rj             202002080040000200000MIQ2M   
md                                            40.209   
billed_amt_sum_effect                      0.0895322   
clm_ln_cnt_effect                                  0   
clm_unit_billed_effect                             0   
proc_code_unusual_effect                    0.875426   
pos_unsual_effect                            95.1702   
em_proc_ind_effect                          0.373051   
mod91_ind_effect                                   0   
modTC_ind_effect                                   0   
mod59_ind_effect                           0.0522271   
high_cst_proc_ind_effect                           0   
emgncy_ind_effect                                  0   

                                                   3  \
servicing_npi                             1033311147   
srk_clm_id_rj             202002171150000300000MICUL   
md                                            30.444   
billed_amt_sum_effect                        1.80003   
clm_ln_cnt_effect                            82.5023   
clm_unit_billed_effect                       1.24819   
proc_code_unusual_effect                    0.492708   
pos_unsual_effect                                  0   
em_proc_ind_effect                          0.059125   
mod91_ind_effect                                   0   
modTC_ind_effect                                   0   
mod59_ind_effect                                   0   
high_cst_proc_ind_effect                           0   
emgncy_ind_effect                                  0   

                                                   4  \
servicing_npi                             1053315820   
srk_clm_id_rj             202003020970000700000MIEJW   
md                                            51.438   
billed_amt_sum_effect                        5.60481   
clm_ln_cnt_effect                            8.49761   
clm_unit_billed_effect                       14.1063   
proc_code_unusual_effect                     1.69524   
pos_unsual_effect                            1.35503   
em_proc_ind_effect                          0.153583   
mod91_ind_effect                                   0   
modTC_ind_effect                           0.0486022   
mod59_ind_effec

In [94]:
diff_np = diff_pd.values
th = 10

In [95]:
# Generating Signals for a Claim
clm_ln = diff_np[1, :]
snpi = clm_ln[0]
print("Claim : {}\n".format(clm_ln))

# Extracting signal effect from claim
clm_ln_se = clm_ln[3:].astype(float)
print("Signal effect : {}\n".format(clm_ln_se))

## Figuring out Reason for high MD score

# Sorting the effect
clm_ln_sort  = np.flip(np.argsort(clm_ln_se))
print(clm_ln_sort)
clm_ln_se_thi = np.argwhere(clm_ln_se > th).flatten()
clm_ln_se_thi = clm_ln_se_thi[0:4]
reason_c_list = []
for rnk in clm_ln_sort:
    if rnk in clm_ln_se_thi:
        reason_c_list.append(mapper[rnk])
reasn_desc_list = []
for rc in reason_c_list:
    rc_avg = "avg_" + rc
    try:
        m_value = str(np.round(mean_data_rdesc_dict[snpi][rc_avg], 2))
        reasn_desc_list.append(reason_desc[rc].replace("(&avg&)", m_value))
    except:
        reasn_desc_list.append(reason_desc[rc])
print("Reason for high MD for {} are {}".format(clm_ln[1], reasn_desc_list))

Claim : ['1023476819' '202002110470000200000MID1H' 8.14 0.0 0.0 0.0
 15.626535626535627 0.0 82.11302211302211 0.0 0.0 0.0 0.0 0.0]

Signal effect : [ 0.          0.          0.         15.62653563  0.         82.11302211
  0.          0.          0.          0.          0.        ]

[ 5  3 10  9  8  7  6  4  2  1  0]
Reason for high MD for 202002110470000200000MID1H are ['E&M code is used in claim.', 'An unusual procedure code is observed. Used 0.3 only in past 3 months']


In [96]:
claim_score = [get_reason_code(clm) for clm in diff_np]
claim_score_df = pd.DataFrame(claim_score, columns = ["servicing_npi", "srk_clm_id_rj", "Score", 
                                                        "Reason 1", "Reason 2", "Reason 3", "Reason 4"])
claim_score_df.head(5).T

0  \
servicing_npi                                         1023005204   
srk_clm_id_rj                         202002070390000200000MJ34B   
Score                                                    6326.25   
Reason 1       Billed Amount of claim is higher than expected...   
Reason 2                                                    None   
Reason 3                                                    None   
Reason 4                                                    None   

                                                               1  \
servicing_npi                                         1023476819   
srk_clm_id_rj                         202002110470000200000MID1H   
Score                                                       8.14   
Reason 1                              E&M code is used in claim.   
Reason 2       An unusual procedure code is observed. Used 0....   
Reason 3                                                    None   
Reason 4                                                    None   

                                                               2  \
servicing_npi                                         1033186200   
srk_clm_id_rj                         202002080040000200000MIQ2M   
Score                                                     40.209   
Reason 1       An unusual place of service is observed. Used ...   
Reason 2                                                    None   
Reason 3                                                    None   
Reason 4                                                    None   

                                                               3  \
servicing_npi                                         1033311147   
srk_clm_id_rj                         202002171150000300000MICUL   
Score                                                     30.444   
Reason 1       No. of claim lines are higher than expected. A...   
Reason 2                                                    None   
Reason 3                                                    None   
Reason 4                                                    None   

                                                               4  
servicing_npi                                         1053315820  
srk_clm_id_rj                         202003020970000700000MIEJW  
Score                                                     51.438  
Reason 1       No. of units billed is higher than expected. A...  
Reason 2                                                    None  
Reason 3                                                    None  
Reason 4                                                    None

In [97]:
claim_score_df.iloc[0,:].values

array(['1023005204', '202002070390000200000MJ34B', 6326.249,
       'Billed Amount of claim is higher than expected. Average Value = 0.01',
       None, None, None], dtype=object)

### NPi Scoring

<b> Freq Flagged claims </b> = # claims above cutoff score / # claims <br>
<b> Severity</b> = Median Score of claims above the cut off score 

In [98]:
# Calculating Frequency of flagged claims 
npi_flg_clm_cnt = flagged_data.groupby("servicing_npi").count().withColumnRenamed("count", "flg_clm_cnt").toPandas()
npi_clm_cnt = scoring_imputed_data.groupby("servicing_npi").count().withColumnRenamed("count", "clm_cnt").toPandas()
npi_scoring = npi_clm_cnt.merge(npi_flg_clm_cnt, on ="servicing_npi", how = "left")
npi_scoring['frequency'] = npi_scoring['flg_clm_cnt']/npi_scoring['clm_cnt']

# Calculating Severity
npi_flg_median_score = flagged_data.groupby("servicing_npi").agg(F.expr('percentile(md, array(0.5))')[0].alias('severity')).toPandas()
npi_scoring = npi_scoring.merge(npi_flg_median_score, on ="servicing_npi", how = "left")

# Calculating NPI Score
npi_scoring = npi_scoring.fillna(0)
npi_scoring = npi_scoring[['servicing_npi', 'frequency', 'severity']]
npi_scoring['score'] = npi_scoring['frequency'] * npi_scoring['severity']
npi_scoring = npi_scoring.sort_values(by ="score", ascending = False).reset_index(drop = True)
npi_scoring.head(10)

servicing_npi  frequency    severity        score
0    1245437078   0.428571  19206.4130  8231.319857
1    1992871321   1.000000   6752.8130  6752.813000
2    1639267636   0.264151  22121.6590  5843.457094
3    1013934983   0.812500   6783.8320  5511.863500
4    1548347321   1.000000   4540.1580  4540.158000
5    1922148253   0.363636  11558.8065  4203.202364
6    1982639589   0.485714   8314.5140  4038.478229
7    1790771293   0.321429  12366.4820  3974.940643
8    1528014404   0.666667   4518.4755  3012.317000
9    1740348580   0.500000   5585.2040  2792.602000